In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import os
import cv2
import random
from matplotlib import pyplot as plt
from tqdm import tqdm

## Mosaic

### 구현 순서

1. 순서대로 이미지를 뽑는다
2. 뽑힌 이미지를 left-top으로 두고 랜덤으로 3개 이미지를 추출
3. 추출된 이미지의 bbox좌표를 right-top -> left-bottom -> right-bottom 순으로 결합
4. 각각 순서대로 bbox의 좌표이동

In [2]:
base_dir = '/opt/ml/detection/dataset/'
data_json = base_dir + 'train.json'

In [3]:
#json 불러오기 + Dataframe형태로 변경
with open (data_json, 'r') as outfile:
    data = json.load(outfile)

df_images = pd.DataFrame(data['images'])
df_annotations = pd.DataFrame(data['annotations'])
df_categories = pd.DataFrame(data['categories'])

In [4]:
mosaic_list = ['lt', 'rt', 'lb', 'rb']
df_images.head(2)

,width,height,file_name,license,flickr_url,coco_url,date_captured,id
0,1024,1024,train/0000.jpg,0,None,None,2020-12-26 14:44:23,0
1,1024,1024,train/0001.jpg,0,None,None,2021-01-10 16:30:39,1


In [5]:
df_images['mosaic_file_name'] = df_images['file_name'].str.replace('train', 'mosaic_train')
df_images.loc[0, 'mosaic_file_name']

'mosaic_train/0000.jpg'

In [6]:
df_images
np.random.seed(2114) #RandomSeed 변경해주세요!

In [7]:
def make_save_image(image_idx_list):
    
    img_list = []
    
    for idx in image_idx_list:       
        img = cv2.imread(base_dir + df_images.loc[idx, 'file_name'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_list.append(img)
        
    
    img1 = np.hstack((img_list[0], img_list[1]))
    img2 = np.hstack((img_list[2], img_list[3]))
    
    img = np.vstack((img1, img2))
    
    plt.imsave(base_dir +df_images.loc[img_idx_list[0], 'mosaic_file_name'], img)

def append_anno(df_param, location):

    anno_bboxes = list(df_param['bbox'])
    
    changed_bboxes = []
    
    if location != 'lt':
        
        if location == 'rt':
    
            for anno_bbox in anno_bboxes:
                x = anno_bbox[0] +1024 
                y = anno_bbox[1]
                w = anno_bbox[2]
                h = anno_bbox[3]
                changed_bboxes.append([x, y, w, h])
                
        elif location == 'lb':
          
            for anno_bbox in anno_bboxes:
                x = anno_bbox[0] 
                y = anno_bbox[1] +1024 
                w = anno_bbox[2]
                h = anno_bbox[3]
                changed_bboxes.append([x, y, w, h])

        elif location == 'rb':
            
            for anno_bbox in anno_bboxes:
                x = anno_bbox[0] +1024
                y = anno_bbox[1] +1024 
                w = anno_bbox[2]
                h = anno_bbox[3]
                changed_bboxes.append([x, y, w, h]) 
         
        df_new = df_param.drop('bbox', axis=1)
        df_new['bbox'] = changed_bboxes
        df_new['location'] = location
    else:
       
        df_new = df_param
        df_new['location'] = location

    return df_new

In [ ]:
df_annotation_new = pd.DataFrame()
for i in tqdm(range(len(df_images))): #df_images
    
    img_idx_list = []
    img_idx = i
    
    original_image_id = df_images.loc[img_idx, 'id']
    
    for location in mosaic_list: #Randomly chosen 3 images
        
        if len(img_idx_list) == 0:
            random_idx = img_idx
        else:
            random_idx = np.random.randint(len(df_images))
        
        ##image concatenation을 위한 index 저장
        img_idx_list.append(random_idx)
    
        ##annotation 저장
        image_id = df_images.loc[random_idx, 'id']
        register_anno = df_annotations.loc[df_annotations['image_id']==image_id]
        register_anno['image_id'] = original_image_id
        df_new = append_anno(register_anno, location)
        df_annotation_new = df_annotation_new.append(df_new, ignore_index=True)

    #image 저장 dataset/mosaic/train/
    df_images['mosaic_file_name'] = df_images['file_name'].str.replace('train', 'mosaic_train')
    make_save_image(img_idx_list)

  0%|          | 0/4883 [00:00<?, ?it/s]/opt/conda/envs/detection/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/detection/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 14%|█▍        | 701/4883 [03:53<22:57,  3.04it/s]

In [ ]:
df_annotations_new = df_annotation_new.sort_values(by='image_id')
df_annotations_new['id'] = np.arange(len(df_annotations_new))
df_annotations_new = df_annotations_new.drop(columns='location')
df_annotations_new

In [ ]:
# 수정된 json 저장
df_new_images = df_images.drop('file_name', axis=1)
df_new_images['width'] = 2048
df_new_images['height'] = 2048
df_new_images = df_new_images.rename(columns={'mosaic_file_name':'file_name'})

In [ ]:
df_annotations_new

In [ ]:
new_images_json = df_new_images.to_json(orient = 'records')
new_images_json = json.loads(new_images_json)

new_anno_json = df_annotations_new.to_json(orient = 'records')
new_anno_json = json.loads(new_anno_json)

new_json = json.dumps({'images':new_images_json, 'annotations':new_anno_json, 'categories':data['categories']})
parsed_json = json.loads(new_json)

file_path = '/opt/ml/detection/dataset/mosaic_train.json'

with open(file_path, 'w') as outfile:
    json.dump(parsed_json, outfile, indent=4)